In [28]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))



df = pd.read_csv(os.path.join(os.getcwd(), "../../data/total_4am.csv"), index_col=0)


In [29]:
from performance import get_performance

def get_results(k, df):

    original_df = df.copy()  # Make a copy of the original DataFrame
    df["range"] = df["high"] - df["low"]
    df["target_price"] = 0
    for i in range(1, len(df)):
        df.loc[i, "target_price"] = df.loc[i-1, "range"] * k + df.loc[i, "open"]

    # df['signal'] = 0  # Default to no position

    
    # for i in range(1, len(df)):
    #     # Buy condition
    #     if df.loc[i, "high"] >= df.loc[i, "target_price"] and signal_before == False:
    #         df.loc[i, 'signal'] = 1
    #         signal_before = True
        

    # # Manage positions with stop loss, take profit, and sell signal
    df['signal'] = 0  # Default to no position
    df['position'] = 0
    df['entry_price'] = np.nan
    df['exit_price'] = np.nan
    holding_position = False
    signal_before = False

    for i in range(1, len(df)):
        if df.loc[i, "high"] >= df.loc[i, "target_price"] and signal_before == False:
            df.loc[i, 'signal'] = 1
            signal_before = True

        if df['signal'].iloc[i] == 1 and not holding_position:
            # Enter position
            df.loc[i, 'position'] = 1
            df.loc[i, 'entry_price'] = df.loc[i, 'target_price']
            holding_position = True

        elif holding_position:
            # Calculate percentage change since entry
            entry_price = df['entry_price'].iloc[i-1]
            current_price = df['close'].iloc[i]
            percent_change = (current_price - entry_price) / entry_price * 100

            if percent_change <= -5:  # Stop loss condition
                # print(f"cond2 on{i}")
                df.loc[i, 'position'] = 0
                df.loc[i, 'exit_price'] = current_price
                holding_position = False
                signal_before = False

            # elif percent_change >= 30:  # Take profit condition
            #     print(f"cond3 on{i}")
            #     df['position'].iloc[i] = 0
            #     df['exit_price'].iloc[i] = current_price
            #     holding_position = False
            else:
                df.loc[i, 'position'] = df.loc[i-1, 'position']
                df.loc[i, 'entry_price'] = df.loc[i-1, 'entry_price']
        else:
            df.loc[i, 'position'] = df.loc[i-1, 'position']

    # Calculate the strategy returns (only when in a long position)
    df['strategy_returns'] = df['position'].shift(1) * df['close'].pct_change()
    df['strategy_returns2'] = df["strategy_returns"]

    for i in range(1, len(df)):
        buy_price = df.loc[i-1, "target_price"]
        buy_price_copy = buy_price
        sell_price = df.loc[i, "close"]
        sell_price_copy = sell_price

        if df.loc[i-1, 'position'] == 1 and df.loc[i-1, 'signal'] == 1:
            # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'])/(df.loc[i-1, 'position'] * 1.002) -1
            buy_price = df.loc[i-1, "target_price"] * 1.002
        if df.loc[i, 'position'] == 0 and df.loc[i-1, 'position'] != 0:
            # df.loc[i, 'strategy_returns2'] = (df.loc[i,'position'] * 0.998)/(df.loc[i-1, 'position']) -1
            buy_price = df.loc[i-1, "close"]
            sell_price = df.loc[i, "close"] * 0.998
        
        if buy_price == buy_price_copy and sell_price == sell_price_copy:            
            continue
        
        # df.loc[i, "strategy_returns"] = sell_price_copy/buy_price_copy -1
        df.loc[i, "strategy_returns2"] = sell_price/buy_price - 1


    df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()

    df["cumulative_returns2"] = (1 + df['strategy_returns2']).cumprod()


    # Calculate the benchmark cumulative returns (buy and hold strategy)
    df['benchmark_returns'] = (1 + df['close'].pct_change()).cumprod()

    # Save the results to a CSV file
    df.to_csv(f"../../results/volatilitybreakout/volatility_breakout_cut_{k}.csv")

    # Get performance metrics and plot the results
    get_performance(df=df, title=f"Volatility Breakout_cut_{k}")

    df = original_df.copy()  # Create a fresh copy for each interval


In [30]:
ks = [k / 10.0 for k in range(1, 10)]
for k in ks:
    get_results(k=k, df=df)

======Investment Summary======
Strategy        : Volatility Breakout_cut_0.1
total_return    : -4.17
cagr            : -1.25
mdd             : 78.16
total_return_w_fee : 23.76
cagr_w_fee      : 6.5
mdd_w_fee       : 73.45
investing_days  : 1236
======Investment Summary======
Strategy        : Volatility Breakout_cut_0.2
total_return    : 89.02
cagr            : 20.69
mdd             : 68.5
total_return_w_fee : 71.2
cagr_w_fee      : 17.21
mdd_w_fee       : 72.38
investing_days  : 1236
======Investment Summary======
Strategy        : Volatility Breakout_cut_0.3
total_return    : 36.38
cagr            : 9.59
mdd             : 70.06
total_return_w_fee : 54.92
cagr_w_fee      : 13.8
mdd_w_fee       : 69.77
investing_days  : 1236
======Investment Summary======
Strategy        : Volatility Breakout_cut_0.4
total_return    : 38.39
cagr            : 10.07
mdd             : 68.79
total_return_w_fee : 44.01
cagr_w_fee      : 11.37
mdd_w_fee       : 69.44
investing_days  : 1236
======Investment S